In [19]:
from numpy import loadtxt, pi
path = loadtxt(r"C:\Users\localadmin.TUD0035314\Documents\GitHub\New_world\robot_trajectory.txt")
poses = []
for i in range(len(path)):
    if path[i][3]!=0:
        if path[i][3] == 1:
            poses.append([path[i][0],path[i][1],path[i][2],pi])
        elif path[i][3] == 3:
            poses.append([path[i][0],path[i][1],path[i][2],0])
        elif path[i][3] == 2:
            poses.append([path[i][0],path[i][1],path[i][2],3*pi/2])
        elif path[i][3] == 4:
            poses.append([path[i][0],path[i][1],path[i][2],pi/2])
        
lposes = len(poses)

In [43]:
from math import tan, atan, sqrt, cos, sin
from numpy import eye
from numpy.linalg import inv
from random import gauss, randint, uniform
import copy

class Particle:
    def __init__(self,pose):
        self.pose = pose
        self.landmark_poses = []
        self.landmark_covariances = []
        
    def number_of_landmarks(self):
        return len(self.landmark_poses)
    
    @staticmethod
    def motion(pose, control, WB, dt):  
        
        control[1]=control[1] % (2*pi)
        angle = ((control[1]+pose[2]*180/pi)%360)*pi/180
        return ([x+y for x,y in zip(pose,[control[0]*dt*cos(angle), 
                                          control[0]*dt*sin(angle), 
                                          control[0]*dt*sin(control[1]*pi/180)/WB])])
    
    def move(self,control,WB,dt):
        self.pose = self.motion(self.pose,control,WB,dt)
        
    def expected_measurement(pose,landmark):
        theta = pose[2]
        phi = landmark[2]
        
        return tan(phi-theta) # Ratio of the accelerations ax/ay       
    
    def measurement_correspondence(self,pose,measurement,number_of_landmarks,Qt_measurment_covariance):
        likelihoods = []
        for i in range(number_of_landmarks):
            likelihoods.append(self.landmark_correspondence(measurement,i,Qt_measurment_covariance))
        return likelihoods
    
    @staticmethod
    def dh_dlandmark(pose,landmark):
        return np.array([[1,0,0],[0,1,0],[0,0,-1/cos(pose[2]-landmark[2])**2]]) # H - Direct odometry measurement and collision
    
    def H_and_Ql_from_measurement(self,landmark_number,Qt_measurement_covariance):
        
        H = self.dh_dlandmark(self.pose,self.landmark[landmark_number])
        Ql = H * self.landmark_covariances[landmark_number] * H.T + Qt_measurement_covariance
        
        return (H, Ql)
        
    def landmark_correspondence_likelihood(self,measurement,landmark_number,Qt_measurement_covariance):
        # For a given measurment and a landmark number, it returns a suitable likelihood value of the correspondence

        landmark = self.landmark_poses[landmark_number]
        phi = landmark[2]
        theta = self.pose[2]
        
        # Prune the landmark which are not in the robot's frame of collision
        if phi == 90 * floor(theta/90) or phi == 90 * ceil(theta/90): 
            zhat = self.expected_measurement(self.pose,landmark)
            H, Ql = self.H_and_Ql_from_measurement(landmark_number,Qt_measurement_covariance)
            dz = measurement - zhat
            
            # Compute likelihood
            l = 0.5
            
        else: 
            l = 0
        
        return l
    
    def intialize_new_landmark(self,measurement,Qt_measurement_covariance):
        
        self.landmark_poses.append([self.pose[0],self.pose[1],self.pose[2]+atan(measurement)]) # Orientation update as FSM
        # We have to modify the orientation of the landmark to multiples of 90 degrees
        Hinv = np.array([[1,0,0],[0,1,0],[0,0,1/(1+measurement**2)]])
        # Remember to add the covariance of the particle at that moment
        self.landmark_covariances.append(Hinv*Qt_measurement_covariance*Hinv.T) 
        
        # Compute likelihood
        l = 0.5
        
        return l
        
    def update_landmark(self,measurment,landmark_number,Qt_measurement_covariance):
        
        mu = self.landmark_poses[landmark_number]
        Sigma = self.landmark_covariances[landmark_number]
        H, Ql = self.H_and_Ql_from_measurement(landmark_number,Qt_measurement_covariance)
        K = Sigma * H.T * inv(Ql)
        
        mu = mu + K*(measurement - expected_measurement(mu,Sigma))
        Sigma = (eye(3) - K*H) * Sigma
        
        return (mu,Sigma)
    
    def update_particle(self,measurement,number_of_landmarks,minimum_correspondence_likelihood,Qt_measurement_covariance):
        
        likelihoods = []
        for i in range(number_of_landmarks):
            likelihoods.append(self.landmark_correspondence_likelihood(measurement,i,Qt_measurement_covariance))
        
        if not likelihoods or max(likelihoods) < minimum_correspondence_likelihood:
            minimum_correspondence_likelihood = self.intialize_new_landmark(measurement,Qt_measurement_covariance)
            return minimum_correspondence_likelihood
        
        else:
            lmax = max(likelihoods)
            lmax_index = likelihoods.index(lmax)
            
            return lmax

class FastSLAM:
    def __init__(self,intial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,
                 x_stddev,y_stddev,control_speed_factor,control_head_factor, sample_time):
        # Particles
        self.particles = intial_particles
        
        # Constants
        self.robot_width = robot_width
        self.minimum_correspondence_likelihood = minimum_correspondence_likelihood
        self.xstddev = xstddev
        self.ystddev = ystddev
        self.measurement_stddev = measurement_stddev
        self.control_speed_factor = control_speed_factor
        self.control_head_factor = control_head_factor
        self.dt = sample_time
        self.WB = robot_width
        
    def predict(self,control):
        # Prediction step of FastSLAM

        speed, head = control

        speed_std = self.control_speed_factor * sqrt(speed) # To be modified

        head_std  = self.control_head_factor * sqrt(359 - head) # To be modified; mirror image of the speed deviation
        
        for p in self.particles:
            speed = gauss(speed,speed_std)
            head = gauss(head,head_std)
            p.move([speed,head],self.WB,self.dt)
    
    def update_and_compute_weights(self,landmarks):
        
        Qt_measurement_covariance = np.diag([self.xstddev ** 2, self.ystddev ** 2,self.measurement_stddev ** 2]) 
        
        weights = []
        print "Particle 1 is",self.particles
        for p in self.particles:
            
            number_of_landmarks = p.number_of_landmarks()
            weight = 1
            
            for measurement in landmarks:
                # Cumulative product of weights for a particle based on the measurements
                weight *= p.update_particle(measurement,number_of_landmarks,self.minimum_correspondence_likelihood,
                                            Qt_measurement_covariance)
                weights.append(weight)
                
            weights.append(weight)
            
        return weights
    
    def resample(self,weights):
        print "weights is",weights
        print "Length is",len(weights)
        # Returns a new set of resampled particles, proportional to their weight
        new_particles = []
        max_weight = max(weights)
        index = randint(0, len(self.particles)-1)
        offset = 0.0
        for i in xrange(len(self.particles)):
            offset += uniform(0, 2.0 * max_weight)
            while offset > weights[index]:
                offset -= weights[index]
                index = (index + 1) % len(weights)
            new_particles.append(copy.deepcopy(self.particles[index]))
            
        return new_particles
    
    def correct(self,landmarks):

        # Correction step of FastSLAM
        weights = self.update_and_compute_weights(landmarks)
        self.particles = self.resample(weights)
        
if __name__ == '__main__':
    
    robot_width = 2
    sample_time = 0.1
    
    minimum_correspondence_likelihood = 1e-3
    xstddev = 10
    ystddev = 10
    measurement_stddev = 100
    
    control_speed_factor = 1
    control_head_factor = 1
    number_of_particles = 100
    
    start_state = poses[0][0:3]
    initial_particles = [copy.copy(Particle(start_state))
                         for _ in xrange(number_of_particles)]
    print "initial particles", initial_particles
    fs = FastSLAM(initial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,xstddev,ystddev,
                 control_speed_factor,control_head_factor, sample_time)
    
    for i in xrange(lposes-1):
        
        control = [100,poses[i+1][2]]
        fs.predict(control)
        
        z = tan(poses[i][3]-poses[i][2])
        measurement = [poses[i][0],poses[i][1],z]

        fs.correct(measurement)

initial particles [<__main__.Particle instance at 0x0000000009677308>, <__main__.Particle instance at 0x00000000096777C8>, <__main__.Particle instance at 0x0000000009677E88>, <__main__.Particle instance at 0x0000000009677408>, <__main__.Particle instance at 0x0000000009677508>, <__main__.Particle instance at 0x0000000009677388>, <__main__.Particle instance at 0x0000000009677088>, <__main__.Particle instance at 0x0000000009677708>, <__main__.Particle instance at 0x00000000096775C8>, <__main__.Particle instance at 0x00000000096778C8>, <__main__.Particle instance at 0x0000000009677E48>, <__main__.Particle instance at 0x0000000009677E08>, <__main__.Particle instance at 0x0000000009677CC8>, <__main__.Particle instance at 0x0000000009677688>, <__main__.Particle instance at 0x0000000009677D48>, <__main__.Particle instance at 0x0000000009677A48>, <__main__.Particle instance at 0x0000000009677C08>, <__main__.Particle instance at 0x0000000009677848>, <__main__.Particle instance at 0x000000000967

IndexError: list index out of range